In [ ]:
# Imports
import cv2
import numpy as np
import os
import matplotlib.pyplot as plt
import glob
from sklearn import svm
from skimage.feature import greycomatrix, greycoprops
from sklearn import metrics
from skimage.feature import local_binary_pattern

In [ ]:
%run Read_Data.ipynb

In [6]:
N_RHO_BINS = 7
N_ANGLE_BINS = 12
N_BINS = N_RHO_BINS * N_ANGLE_BINS
BIN_SIZE = 360 // N_ANGLE_BINS
R_INNER = 5.0
R_OUTER = 35.0
APPROX_POLY_FACTOR = 0.01
K_S = np.arange(3, 6)
np.seterr(divide = 'ignore')

{'divide': 'ignore', 'over': 'warn', 'under': 'ignore', 'invalid': 'warn'}

In [7]:
def get_cold_feature(image):
	image = cv2.GaussianBlur(image,(3,3),0)
	bw_image = cv2.threshold(image, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)[1]

	contours, _= cv2.findContours(
				bw_image, cv2.RETR_TREE, 
				cv2.CHAIN_APPROX_NONE
				)
	contours = sorted(contours, key=cv2.contourArea, reverse=True)[1:]

	rho_bins_edges = np.log10(np.linspace(R_INNER, R_OUTER, N_RHO_BINS))
	feature_vectors = np.zeros((len(K_S), N_BINS))

	for j, k in enumerate(K_S):
		hist = np.zeros((N_RHO_BINS, N_ANGLE_BINS))
		for cnt in contours:
			epsilon = APPROX_POLY_FACTOR * cv2.arcLength(cnt,True)
			cnt = cv2.approxPolyDP(cnt,epsilon,True)
			n_pixels = len(cnt)
			
			point_1s = np.array([point[0] for point in cnt])
			x1s, y1s = point_1s[:, 0], point_1s[:, 1]
			point_2s = np.array([cnt[(i + k) % n_pixels][0] for i in range(n_pixels)])
			x2s, y2s = point_2s[:, 0], point_2s[:, 1]
			
			thetas = np.degrees(np.arctan2(y2s - y1s, x2s - x1s) + np.pi)
			rhos = np.sqrt((y2s - y1s) ** 2 + (x2s - x1s) ** 2)
			rhos_log_space = np.log10(rhos)
			
			quantized_rhos = np.zeros(rhos.shape, dtype=int)
			for i in range(N_RHO_BINS):
				quantized_rhos += (rhos_log_space < rho_bins_edges[i])
				
			for i, r_bin in enumerate(quantized_rhos):
				theta_bin = int(thetas[i] // BIN_SIZE) % N_ANGLE_BINS
				hist[r_bin - 1, theta_bin] += 1
		normalised_hist = hist / hist.sum()
		feature_vectors[j] = normalised_hist.flatten()
	feature_vectors = np.nan_to_num(feature_vectors)
	feature_vector=feature_vectors.flatten()
	return feature_vector

(252,)


In [ ]:
x_train = np.array([get_cold_feature((img)) for img in tqdm(x_train)])
x_test = np.array([get_cold_feature((img)) for img in tqdm(x_test)])

In [ ]:
x_train_male = x_train[y_train == 0]
x_train_female = x_train[y_train == 1]
plt.scatter(x_train_male[:,2], x_train_male[:,4], color='red', s=4)
plt.scatter(x_train_female[:,2], x_train_female[:,4], color='blue', s=4)
plt.title('Linearly separable data')
plt.xlabel('feature 1')
plt.ylabel('feature 2')
plt.show()